# benchmark different energy computation methods to see which one is the fastests

In [1]:
import timeit
import numpy as np

In [64]:
L = 8

In [65]:
state = np.random.rand(L**2)
new_state = lambda: np.random.rand(L**2)

number = 100_000
stime = np.array(timeit.repeat(lambda: np.random.rand(L**2), number=number))
nstime = np.array(timeit.repeat(lambda: new_state(), number=number))
print(f"{stime=} {nstime=} {nstime-stime}")

stime=array([0.1308815, 0.1279721, 0.1300804, 0.139281 , 0.1333082]) nstime=array([0.1364531, 0.1490472, 0.1517211, 0.1453757, 0.1658446]) [0.0055716 0.0210751 0.0216407 0.0060947 0.0325364]


In [30]:
def state_to_lattice(state):
    lattice = np.reshape(2 * np.pi * state, (L, L))
    return lattice


number = 1_000_000
np.array(timeit.repeat(lambda: state_to_lattice(new_state()), number=number)) / number

array([6.7391841e-06, 7.4477916e-06, 7.1829801e-06, 6.7562989e-06,
       6.2980928e-06])

In [59]:
def compute_energy(state):
    """
    Computes energy of the current state
    """
    # J=0 except for nearest neighbor
    lattice = state_to_lattice(state)
    energy = (
        -sum(
            np.cos(lattice[i, j] - lattice[i - 1, j])
            + np.cos(lattice[i, i] - lattice[i, j - 1])
            for i in range(L)
            for j in range(L)
        )
    ) / L**2
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy(new_state()), number=number))

array(319.9243684)

In [ ]:
def compute_energy2(state):
    lattice = state_to_lattice(state)
    energy = (
        -np.sum(
            np.cos(lattice[i, j] - lattice[i - 1, j])
            + np.cos(lattice[i, i] - lattice[i, j - 1])
            for i in range(L)
            for j in range(L)
        )
        / L**2
    )
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy2(new_state()), number=number))

In [61]:
def compute_energy_roll(state):
    lattice = state_to_lattice(state)
    energy = (
        -np.sum(
            np.cos(lattice - np.roll(lattice, -1, axis=0))
            + np.cos(lattice - np.roll(lattice, -1, axis=1))
        )
        / L**2
    )
    return energy


number = 100_000
np.array(timeit.timeit(lambda: compute_energy_roll(new_state()), number=number))

array(11.4378669)

In [72]:
number = 200_000
timeit.repeat(lambda: (new_state() + new_state()) % 1, number=number)

[1.240578099997947,
 1.3364106000080938,
 1.1644328999973368,
 1.1640502000082051,
 1.2120015999971656]